# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset

import tensorflow as tf
from tensorflow import keras

c:\Users\lenovo\anaconda3\envs\Sepehr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataframe1 = pd.read_csv('CompleteDataSet.csv', dtype="str")

dataset = dataframe1.values.tolist()

new_dataset = np.array(dataset)

In [3]:
X = []
y = []

for i in range(1,4):
    # Trial
    for j in range(1,12):
        # Activity
        for k in range(1,18):
            # Subject
            z = []
            # Each Video
            for n in range(1,len(new_dataset)-1):
                if (float(new_dataset[n][45]) == i) & (float(new_dataset[n][44]) == j) & (float(new_dataset[n][43]) == k):
                    a = new_dataset[n,15:18]
                    a = a.astype(np.float32)
                    b = new_dataset[n,29:32]
                    b = b.astype(np.float32)
                    c = np.concatenate((a,b), axis = 0)
                    z.append(c)
            X.append(z)
            y.append(j)

In [4]:
X.pop(551)
X.pop(364)
y.pop(551)
y.pop(364)


11

In [5]:
counter = 0
max = 0
min = 1000
for i in range(len(X)):
    counter += len(X[i])
    if len(X[i]) == 0:
        print(i)
    if len(X[i]) > max:
        max = len(X[i])
    if len(X[i]) < min:
        min = len(X[i])

print(f"Avg: {counter/len(X)}\nMin: {min}\nMax: {max}")

Avg: 527.1502683363149
Min: 140
Max: 1145


In [6]:
def to_np_array(X):
    for i in range(len(X)):
        X[i] = np.array(X[i])
        for j in range(len(X[i])):
            X[i][j] = np.array(X[i][j])
    X = np.array(X)
    return X

In [7]:
for i in range(len(y)):
  if (y[i] >= 6) & (y[i] <= 11):
    y[i] = 0
  elif (y[i] >= 1) & (y[i] <= 5):
    y[i] = 1
  else:
    print(i)

In [8]:
X_train = []
X_test = []
y_train = []
y_test = []

for i in range(1,12):
    counter = 0
    for j in range(len(y)):
        if (i == y[j]) & (counter < 11):
            X_test.append(X[j])
            y_test.append(y[j])
            counter += 1
        elif (i == y[j]) & (counter >= 11):
            X_train.append(X[j])
            y_train.append(y[j])
            counter += 1

X_train = to_np_array(X_train)
X_test = to_np_array(X_test)
y_train = np.array(y_train, dtype=np.uint8)
y_test = np.array(y_test, dtype=np.uint8)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_2920\3925073110.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


In [9]:
print(type(X_train))
print(type(X_train[0]))
print(type(X_train[0][0]))
print(type(X_train[0][0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


# PyTorch

In [10]:
X_train_tensor = torch.zeros((len(X_train),max, 6))
X_test_tensor = torch.zeros((len(X_test),max, 6))
y_train_tensor = torch.Tensor(y_train)
y_test_tensor = torch.Tensor(y_test)

for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        for k in range(len(X_train[i][j])):
            X_train_tensor[i][j][k] = float(X_train[i][j][k])
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        for k in range(len(X_test[i][j])):
            X_test_tensor[i][j][k] = float(X_test[i][j][k])

print(X_train_tensor.shape)
print(X_test_tensor.shape)

torch.Size([244, 1145, 6])
torch.Size([11, 1145, 6])


In [11]:
train_dataset = []
for i in range(len(X_train_tensor)):
    a = (X_train_tensor[i], y_train_tensor[i])
    train_dataset.append(a)
test_dataset = []
for i in range(len(X_test_tensor)):
    a = (X_test_tensor[i], y_test_tensor[i])
    test_dataset.append(a)

In [12]:
# Device configuration
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 2
num_epochs = 2
batch_size = 17
learning_rate = 0.001

input_size = 6
sequence_length = 28
hidden_size = 600
num_layers = 2

# MNIST dataset 
# train_dataset = torchvision.datasets.MNIST(root='./data', 
#                                            train=True, 
#                                            transform=transforms.ToTensor(),  
#                                            download=True)

# test_dataset = torchvision.datasets.MNIST(root='./data', 
#                                           train=False, 
#                                           transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.4)

        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, num_classes)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        # out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.relu(out)
        # out = self.fc2(out)
        # out = self.relu(out)
        out = self.fc1(out)
        out = self.softmax(out)
        # out: (n, 10)
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        images = images
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        # images = images.to(device)
        # labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {len(images)} test images: {acc} %')

Epoch [1/2], Step [1/15], Loss: 0.7012
Epoch [1/2], Step [2/15], Loss: 0.6624
Epoch [1/2], Step [3/15], Loss: 0.6074
Epoch [1/2], Step [4/15], Loss: 0.4624
Epoch [1/2], Step [5/15], Loss: 0.3136
Epoch [1/2], Step [6/15], Loss: 0.3133


In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

example = torch.Tensor(batch_size, max, input_size)

traced_script_module = torch.jit.trace(model, example)

output = traced_script_module(torch.ones(batch_size, max, input_size))

sm = torch.jit.script(model)

traced_script_module.save("LSTM.pt")